# training yolov8

In [ ]:
from ultralytics import YOLO

# Load model klasifikasi ringan (n = nano)
model = YOLO('yolov8n-cls.pt')

# Training ulang pada folder PlantVillage1
model.train(
    data='PlantVillage1',  # folder dataset
    epochs=10,              # jumlah epoch
    imgsz=224               # ukuran gambar input
)


# Grafik Precission Recall F1 score !


In [ ]:
from ultralytics import YOLO
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import os

# Load model terbaik
model = YOLO('runs/classify/train/weights/best.pt')
class_names = list(model.model.names.values())
test_dir = 'PlantDoc-Dataset'

y_true = []
y_pred = []

# Loop gambar di setiap kelas
for class_index, class_name in enumerate(class_names):
    class_path = os.path.join(test_dir, class_name)
    if not os.path.isdir(class_path): continue
    for file in os.listdir(class_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(class_path, file)
            results = model(img_path)
            pred_index = results[0].probs.top1

            y_true.append(class_index)
            y_pred.append(pred_index)

# Hitung metrik
report_dict = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
accuracy = accuracy_score(y_true, y_pred)

print(f"Akurasi Keseluruhan: {accuracy * 100:.2f}%")

# Simpan teks evaluasi ke file
from sklearn.metrics import classification_report
with open('hasil_evaluasi.txt', 'w') as f:
    f.write(f"Akurasi: {accuracy * 100:.2f}%\n\n")
    f.write(classification_report(y_true, y_pred, target_names=class_names))

# Ambil metrik untuk grafik
precision = [report_dict[cls]['precision'] for cls in class_names]
recall = [report_dict[cls]['recall'] for cls in class_names]
f1 = [report_dict[cls]['f1-score'] for cls in class_names]

x = np.arange(len(class_names))
width = 0.25

# Plot
plt.figure(figsize=(12, 6))
plt.bar(x - width, precision, width, label='Precision')
plt.bar(x, recall, width, label='Recall')
plt.bar(x + width, f1, width, label='F1-score')

plt.xlabel('Kelas')
plt.ylabel('Skor')
plt.title('Precision, Recall, dan F1-score per Kelas')
plt.xticks(ticks=x, labels=class_names, rotation=45, ha='right')
plt.ylim(0, 1.05)
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.show()


# Ini kalau ada gambar
gambarnya masukan ke test_image

In [ ]:
import os

# Load model deteksi (bukan klasifikasi)
model = YOLO('runs/classify/train/weights/best.pt')

# Path folder gambar uji
test_folder = 'test_closeimg'

# Loop dan deteksi semua gambar
for file in os.listdir(test_folder):
    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(test_folder, file)
        results = model(path, save=True)  # akan menyimpan hasil deteksi secara otomatis di runs/detect


# Ngelihatin gambar di folder apa aja Random

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

# Path dataset
dataset_dir = 'PlantVillage1_split/val'

# Ambil semua folder kelas
class_folders = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

# Pilih 4 kelas secara acak
selected_classes = random.sample(class_folders, 4)

# Tampilkan 1 gambar dari masing-masing kelas
plt.figure(figsize=(12, 4))

for i, class_name in enumerate(selected_classes):
    class_path = os.path.join(dataset_dir, class_name)
    image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if image_files:
        selected_image = random.choice(image_files)
        img_path = os.path.join(class_path, selected_image)
        img = mpimg.imread(img_path)

        plt.subplot(2, 2, i + 1)
        plt.imshow(img)
        plt.title(f"{class_name}\n{selected_image}", fontsize=8)
        plt.axis('off')
    else:
        print(f"⚠️ Tidak ada gambar di folder {class_name}")

plt.tight_layout()
plt.show()
